# Spark LLM Assistant

## Initialization

In [1]:
from langchain.chat_models import ChatOpenAI
from spark_llm import SparkLLMAssistant

llm = ChatOpenAI(model_name='gpt-4') # using gpt-4 can achieve better results
assistant=SparkLLMAssistant(llm=llm, verbose=True)
assistant.activate() # active partial functions for Spark DataFrame

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 01:15:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Example 1: Auto sales by brand in US 2022

In [2]:
# Search and ingest web content into a DataFrame
auto_df = assistant.create_df("2022 USA national auto sales by brand")
auto_df.show()

INFO: Parsing URL: https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand

INFO: SQL query for the ingestion:

INFO: CREATE OR REPLACE TEMP VIEW auto_sales_2022 AS SELECT * FROM VALUES
('Toyota', 1849751, -9),
('Ford', 1767439, -2),
('Chevrolet', 1502389, 6),
('Honda', 881201, -33),
('Hyundai', 724265, -2),
('Kia', 693549, -1),
('Jeep', 684612, -12),
('Nissan', 682731, -25),
('Subaru', 556581, -5),
('Ram Trucks', 545194, -16),
('GMC', 517649, 7),
('Mercedes-Benz', 350949, 7),
('BMW', 332388, -1),
('Volkswagen', 301069, -20),
('Mazda', 294908, -11),
('Lexus', 258704, -15),
('Dodge', 190793, -12),
('Audi', 186875, -5),
('Cadillac', 134726, 14),
('Chrysler', 112713, -2),
('Buick', 103519, -42),
('Acura', 102306, -35),
('Volvo', 102038, -16),
('Mitsubishi', 102037, -16),
('Lincoln', 83486, -4),
('Porsche', 70065, 0),
('Genesis', 56410, 14),
('INFINITI', 46619, -20),
('MINI', 29504, -1),
('Alfa Romeo', 12845, -30),
('Maserati', 6413, -10),
('Bentley', 3975, 0),
('Lamborghin

+-------------+-------------+-------+
|        brand|us_sales_2022|vs_2021|
+-------------+-------------+-------+
|       Toyota|      1849751|     -9|
|         Ford|      1767439|     -2|
|    Chevrolet|      1502389|      6|
|        Honda|       881201|    -33|
|      Hyundai|       724265|     -2|
|          Kia|       693549|     -1|
|         Jeep|       684612|    -12|
|       Nissan|       682731|    -25|
|       Subaru|       556581|     -5|
|   Ram Trucks|       545194|    -16|
|          GMC|       517649|      7|
|Mercedes-Benz|       350949|      7|
|          BMW|       332388|     -1|
|   Volkswagen|       301069|    -20|
|        Mazda|       294908|    -11|
|        Lexus|       258704|    -15|
|        Dodge|       190793|    -12|
|         Audi|       186875|     -5|
|     Cadillac|       134726|     14|
|     Chrysler|       112713|     -2|
+-------------+-------------+-------+
only showing top 20 rows



In [3]:
auto_df.llm.plot()

INFO: ```
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming df is a valid PySpark DataFrame
pdf = df.toPandas()

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bars for US sales in 2022
fig.add_trace(go.Bar(x=pdf["brand"], y=pdf["us_sales_2022"], name="US Sales in 2022"), secondary_y=False)

# Add line for comparison to 2021
fig.add_trace(go.Scatter(x=pdf["brand"], y=pdf["vs_2021"], name="vs 2021", mode="lines+markers"), secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Brand")

# Set y-axes titles
fig.update_yaxes(title_text="US Sales in 2022", secondary_y=False)
fig.update_yaxes(title_text="vs 2021", secondary_y=True)

# Add title and show the plot
fig.update_layout(title="US Auto Sales Analysis by Brand - 2022 vs 2021")
fig.show()
```


In [ ]:
auto_df.llm.plot("pie char for top 5 brands and the others' market shares")

In [ ]:
# Apply transforms to a Dataframe
auto_top_growth_df=auto_df.llm.transform("top brand with the highest growth")
auto_top_growth_df.show()

In [ ]:
# Explain what a DataFrame is retrieving.
auto_top_growth_df.llm.explain()

In [ ]:
auto_top_growth_df.llm.verify("expect sales change percentage to be between -100 to 100")

## Example 2: USA Presidents

In [ ]:
# You can also specify the expected columns for the ingestion.
df=assistant.create_df("USA presidents", ["president", "vice_president"])
df.show()

In [ ]:
presidents_who_were_vp = df.llm.transform("presidents who were also vice presidents")
presidents_who_were_vp.show()

In [ ]:
presidents_who_were_vp.llm.explain()

In [ ]:
presidents_who_were_vp.llm.verify("expect no NULL values")

# Example 3: Top 10 tech companies

In [ ]:
# Search and ingest web content into a DataFrame
company_df=assistant.create_df("Top 10 tech companies by market cap", ['company', 'cap', 'country'])
company_df.show()

In [ ]:
us_company_df=company_df.llm.transform("companies in United States")
us_company_df.show()

In [ ]:
us_company_df.llm.explain()

In [ ]:
us_company_df.llm.verify("expect all company names to be unique")

## Example 4: Ingestion from a URL
Instead of searching for the web page, you can also ask the assistant to ingest from a URL.

In [ ]:
best_albums_df = assistant.create_df('https://time.com/6235186/best-albums-2022/', ["album", "artist", "year"])
best_albums_df.show()

In [ ]:
best_albums_df.llm.verify("expect each year to be 2022")

## Example 5: UDF Generation

You can also ask the assistant to generate code for a Spark UDF by providing.

In [ ]:
@assistant.udf
def convert_grades(grade_percent: float) -> str:
    """Convert the grade percent to a letter grade using standard cutoffs"""
    ...

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.udf.register("convert_grades", convert_grades)
percentGrades = [(1, 97.8), (2, 72.3), (3, 81.2)]
df = spark.createDataFrame(percentGrades, ["student_id", "grade_percent"])
df.selectExpr("student_id", "convert_grades(grade_percent)").show()

# Cache
The SparkLLMAssistant supports a simple in-memory and persistent cache system. It keeps an in-memory staging cache, which gets updated for LLM and web search results. The staging cache can be persisted through the commit() method. Cache lookup is always performed on the persistent cache only.

In [ ]:
assistant.commit()